In [70]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

StatementMeta(smallsparkpool, 0, 71, Finished, Available, Finished)

In [71]:
%%pyspark
df = spark.read.load('abfss://files@datalakemwdvy04.dfs.core.windows.net/products/products.csv', format='csv'
# If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(smallsparkpool, 0, 72, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6a4781b1-f390-486e-bc17-22f2a9a43aca)

In [72]:
 delta_table_path = "/delta/products-delta"
 df.write.format("delta").mode("overwrite").save(delta_table_path)

StatementMeta(smallsparkpool, 0, 73, Finished, Available, Finished)

# Update products table

In [4]:
 from delta.tables import *
 from pyspark.sql.functions import *

 # Create a deltaTable object
 deltaTable = DeltaTable.forPath(spark, delta_table_path)

 # Update the table (reduce price of product 771 by 10%)
 deltaTable.update(
     condition = "ProductID == 771",
     set = { "ListPrice": "ListPrice * 0.9" })

 # View the updated data as a dataframe
 deltaTable.toDF().show(10)

StatementMeta(sparkmwdvy04, 0, 5, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [5]:
 new_df = spark.read.format("delta").load(delta_table_path)
 new_df.show(10)

StatementMeta(sparkmwdvy04, 0, 6, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [6]:
display(new_df.limit(10))

StatementMeta(sparkmwdvy04, 0, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a0716cc7-6280-4648-b139-eb273f99403a)

In [7]:
 new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
 new_df.show(10)

StatementMeta(sparkmwdvy04, 0, 8, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [8]:
 deltaTable.history(10).show(20, False, True)

StatementMeta(sparkmwdvy04, 0, 9, Finished, Available, Finished)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 1                                                                                                                                                         
 timestamp           | 2024-08-25 05:23:48                                                                                                                                       
 userId              | null                                                                                                                                                      
 userName            | null                                                                                                                                                      
 operation           | UPDATE                                                                                 

In [9]:
 spark.sql("CREATE DATABASE AdventureWorks")
 spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

StatementMeta(sparkmwdvy04, 0, 10, Finished, Available, Finished)

+----------------------------+-----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                              |comment|
+----------------------------+-----------------------------------------------------------------------+-------+
|ProductID                   |string                                                                 |       |
|ProductName                 |string                                                                 |       |
|Category                    |string                                                                 |       |
|ListPrice                   |string                                                                 |       |
|                            |                                                                       |       |
|# Partitioning              |                                                                       |       |
|

In [10]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsExternal;

StatementMeta(sparkmwdvy04, 0, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [11]:
 df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

StatementMeta(sparkmwdvy04, 0, 13, Finished, Available, Finished)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                       |comment|
+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|ProductID                   |string                                                                                                                          |       |
|ProductName                 |string                                                                                                                          |       |
|Category                    |string                                                                                                                          | 

In [12]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsManaged;

StatementMeta(sparkmwdvy04, 0, 15, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [13]:
 %%sql

 USE AdventureWorks;

 SHOW TABLES;

StatementMeta(sparkmwdvy04, 0, 17, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 2 rows and 3 fields>

In [14]:
 %%sql

 USE AdventureWorks;

 DROP TABLE IF EXISTS ProductsExternal;
 DROP TABLE IF EXISTS ProductsManaged;

StatementMeta(sparkmwdvy04, 0, 20, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [15]:
 %%sql

 USE AdventureWorks;

 CREATE TABLE Products
 USING DELTA
 LOCATION '/delta/products-delta';

StatementMeta(sparkmwdvy04, 0, 22, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [16]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM Products;

StatementMeta(sparkmwdvy04, 0, 24, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [17]:
 from notebookutils import mssparkutils
 from pyspark.sql.types import *
 from pyspark.sql.functions import *

 # Create a folder
 inputPath = '/data/'
 mssparkutils.fs.mkdirs(inputPath)

 # Create a stream that reads data from the folder, using a JSON schema
 jsonSchema = StructType([
 StructField("device", StringType(), False),
 StructField("status", StringType(), False)
 ])
 iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

 # Write some event data to the folder
 device_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''
 mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
 print("Source stream created...")

StatementMeta(sparkmwdvy04, 0, 25, Finished, Available, Finished)

Source stream created...


In [18]:
 # Write the stream to a delta table
 delta_stream_table_path = '/delta/iotdevicedata'
 checkpointpath = '/delta/checkpoint'
 deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
 print("Streaming to delta sink...")

StatementMeta(sparkmwdvy04, 0, 26, Finished, Available, Finished)

Streaming to delta sink...


In [19]:
 # Read the data in delta format into a dataframe
 df = spark.read.format("delta").load(delta_stream_table_path)
 display(df)

StatementMeta(sparkmwdvy04, 0, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7c4cdfcc-f948-46a1-a873-3a555c1ee84f)

In [20]:
 # create a catalog table based on the streaming sink
 spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

StatementMeta(sparkmwdvy04, 0, 28, Finished, Available, Finished)

DataFrame[]

In [21]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparkmwdvy04, 0, 29, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [22]:
 # Add more data to the source stream
 more_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''

 mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(sparkmwdvy04, 0, 30, Finished, Available, Finished)

True

In [23]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(sparkmwdvy04, 0, 31, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [24]:
 deltastream.stop()

StatementMeta(sparkmwdvy04, 0, 32, Finished, Available, Finished)

In [25]:
 deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
 print("Streaming to delta sink...")

StatementMeta(sparkmwdvy04, 0, 33, Finished, Available, Finished)

Streaming to delta sink...


In [26]:
 # Add more data to the source stream
 more_data2 = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev3","status":"ok"}
 {"device":"Dev3","status":"ok"}'''

 mssparkutils.fs.put(inputPath + "more-data2.txt", more_data2, True)

StatementMeta(sparkmwdvy04, 0, 34, Finished, Available, Finished)

True

# Salesforce

In [87]:
 spark.sql("CREATE DATABASE IF NOT EXISTS salesforce")

StatementMeta(smallsparkpool, 0, 88, Finished, Available, Finished)

DataFrame[]

In [96]:
def to_snake_case(col_name):
    # Special case for 'CompanyEXTID'
    if col_name.lower() in ["CompanyEXTID".lower(), "CompanEXTID".lower()]:
        return "company_ext_id"
    # General conversion for other columns
    return col_name.lower().replace(" ", "_")

StatementMeta(smallsparkpool, 0, 97, Finished, Available, Finished)

## Accounts

In [88]:
%%pyspark
account_df = spark.read.load('abfss://files@datalakemwdvy04.dfs.core.windows.net/data/Accounts.csv', format='csv', header=True, lineSep='\r')
display(df.limit(10))

StatementMeta(smallsparkpool, 0, 89, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f42f9be8-16b5-42f4-87e5-5b9773636fc5)

In [89]:
account_df.columns

StatementMeta(smallsparkpool, 0, 90, Finished, Available, Finished)

['CompanyEXTID', 'Company Name', 'City', 'State']

StatementMeta(smallsparkpool, 0, 91, Finished, Available, Finished)

In [91]:
# Apply the function to rename columns
account_df = account_df.toDF(*[to_snake_case(col) for col in account_df.columns])

# Display the renamed columns
account_df.columns

StatementMeta(smallsparkpool, 0, 92, Finished, Available, Finished)

['company_ext_id', 'company_name', 'city', 'state']

In [92]:
account_df.printSchema()

StatementMeta(smallsparkpool, 0, 93, Finished, Available, Finished)

root
 |-- company_ext_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)



In [94]:
account_df = account_df.withColumn("company_ext_id", F.col("company_ext_id").cast(T.IntegerType()))

StatementMeta(smallsparkpool, 0, 95, Finished, Available, Finished)

In [95]:
account_df.printSchema()

StatementMeta(smallsparkpool, 0, 96, Finished, Available, Finished)

root
 |-- company_ext_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)



In [97]:
#  account_df.write.format("delta").mode("overwrite").save("/delta/salesforce/accounts")

StatementMeta(smallsparkpool, 0, 98, Finished, Available, Finished)

In [98]:
account_df.write.format("delta").saveAsTable("salesforce.accounts")
spark.sql("DESCRIBE EXTENDED salesforce.accounts").show(truncate=False)

StatementMeta(smallsparkpool, 0, 99, Finished, Available, Finished)

+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                            |comment|
+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|company_ext_id              |int                                                                                                                  |null   |
|company_name                |string                                                                                                               |null   |
|city                        |string                                                                                                               |null   |
|state                       |string                      

## Contacts

In [99]:
%%pyspark
contact_df = spark.read.load('abfss://files@datalakemwdvy04.dfs.core.windows.net/data/Contacts.csv', format='csv', header=True, lineSep='\r')
display(contact_df.limit(10))

StatementMeta(smallsparkpool, 0, 100, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1b049084-1a3f-4ccd-a5b9-7a30aa5f906e)

In [103]:
contact_df.columns

StatementMeta(smallsparkpool, 0, 104, Finished, Available, Finished)

['companyEXTID', 'first_name', 'last_name', 'email', 'country']

In [104]:
contact_df = contact_df.toDF(*[to_snake_case(col) for col in contact_df.columns])

# Display the renamed columns
contact_df.columns

StatementMeta(smallsparkpool, 0, 105, Finished, Available, Finished)

['company_ext_id', 'first_name', 'last_name', 'email', 'country']

In [105]:
contact_df.printSchema()

StatementMeta(smallsparkpool, 0, 106, Finished, Available, Finished)

root
 |-- company_ext_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)



In [107]:
contact_df = contact_df.withColumn("company_ext_id", F.col("company_ext_id").cast(T.IntegerType()))

StatementMeta(smallsparkpool, 0, 108, Finished, Available, Finished)

In [109]:
# contact_df.write.format("delta").save("/delta/salesforce/contacts")

StatementMeta(smallsparkpool, 0, 110, Finished, Available, Finished)

In [110]:
contact_df.write.format("delta").saveAsTable("salesforce.contacts")
spark.sql("DESCRIBE EXTENDED salesforce.contacts").show(truncate=False)

StatementMeta(smallsparkpool, 0, 111, Finished, Available, Finished)

+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                            |comment|
+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|company_ext_id              |int                                                                                                                  |null   |
|first_name                  |string                                                                                                               |null   |
|last_name                   |string                                                                                                               |null   |
|email                       |string                      

## Leads

In [115]:
%%pyspark
lead_df = spark.read.load('abfss://files@datalakemwdvy04.dfs.core.windows.net/data/Leads.csv', format='csv', header=True, lineSep='\r')
display(lead_df.limit(10))

StatementMeta(smallsparkpool, 0, 116, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4e066b27-45f1-4d92-9b03-ca3aa8caf305)

In [116]:
lead_df = lead_df.toDF(*[to_snake_case(col) for col in lead_df.columns])

# Display the renamed columns
lead_df.columns

StatementMeta(smallsparkpool, 0, 117, Finished, Available, Finished)

['first_name', 'last_name', 'email', 'company', 'lead_status']

In [117]:
# lead_df.write.format("delta").save("/delta/salesforce/leads")

StatementMeta(smallsparkpool, 0, 118, Finished, Available, Finished)

In [118]:
lead_df.write.format("delta").saveAsTable("salesforce.leads")
spark.sql("DESCRIBE EXTENDED salesforce.leads").show(truncate=False)

StatementMeta(smallsparkpool, 0, 119, Finished, Available, Finished)

+----------------------------+------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                         |comment|
+----------------------------+------------------------------------------------------------------------------------------------------------------+-------+
|first_name                  |string                                                                                                            |null   |
|last_name                   |string                                                                                                            |null   |
|email                       |string                                                                                                            |null   |
|company                     |string                                        

## Opportunities

In [123]:
%%pyspark
opportunite_df = spark.read.load('abfss://files@datalakemwdvy04.dfs.core.windows.net/data/Opportunites.csv', format='csv', header=True, lineSep='\r')
display(opportunite_df.limit(10))

StatementMeta(smallsparkpool, 0, 124, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0e3fd926-563b-42ed-afff-ce0b40ca0fe9)

In [124]:
opportunite_df = opportunite_df.toDF(*[to_snake_case(col) for col in opportunite_df.columns])

# Display the renamed columns
opportunite_df.columns

StatementMeta(smallsparkpool, 0, 125, Finished, Available, Finished)

['company_ext_id',
 'amount',
 'project_name',
 'oppurtunity_name',
 'stage',
 'close_date']

In [125]:
opportunite_df = opportunite_df.withColumn("company_ext_id", F.col("company_ext_id").cast(T.IntegerType())) \
       .withColumn("close_date", F.to_date(F.col("close_date"), "M/d/yy")) \
       .withColumn("amount", F.regexp_replace(F.col("amount"), "[$,]", "").cast(T.FloatType()))


# Display the updated schema to verify changes
opportunite_df.printSchema()

StatementMeta(smallsparkpool, 0, 126, Finished, Available, Finished)

root
 |-- company_ext_id: integer (nullable = true)
 |-- amount: float (nullable = true)
 |-- project_name: string (nullable = true)
 |-- oppurtunity_name: string (nullable = true)
 |-- stage: string (nullable = true)
 |-- close_date: date (nullable = true)



In [126]:
# opportunite_df.write.format("delta").save("/delta/salesforce/opportunites")

StatementMeta(smallsparkpool, 0, 127, Finished, Available, Finished)

In [127]:
opportunite_df.write.format("delta").saveAsTable("salesforce.opportunites")
spark.sql("DESCRIBE EXTENDED salesforce.opportunites").show(truncate=False)

StatementMeta(smallsparkpool, 0, 128, Finished, Available, Finished)

+----------------------------+-------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                |comment|
+----------------------------+-------------------------------------------------------------------------------------------------------------------------+-------+
|company_ext_id              |int                                                                                                                      |null   |
|amount                      |float                                                                                                                    |null   |
|project_name                |string                                                                                                                   |null   |
|oppurtunity_name            |stri